In [1]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [2]:
import pandas as pd
import numpy as np
from typing import List, Optional

import statsmodels.api as sm

# plt.style.use('dark_background')
random_state=42

In [3]:
tesis = pd.read_csv("Datos/tesis_final.csv")
tesis["Adherencia_Perfecta"] = tesis["Adherencia_Total"] == 1
tesis.head()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC,Adherencia_Acumulada,Adherencia_Total,TAS_Media_Acumulada,Adherencia_Perfecta
0,4026,0,119.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,119.000000,True
1,4026,1,127.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,123.000000,True
2,4026,2,140.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,128.666667,True
3,4026,3,146.712710,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,133.178178,True
4,4026,4,177.708084,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,142.084159,True


In [4]:
tesis.isnull().sum()

idPaciente                0
tpo_programa              0
TAS                       0
Adherencia                0
Peso                     63
Altura                   63
IMC                      63
DBT                     126
Sexo                      0
Edad                    119
Fuma                    126
ant_HTA                   0
tas_basal                 0
ICC                     266
Adherencia_Acumulada      0
Adherencia_Total          0
TAS_Media_Acumulada       0
Adherencia_Perfecta       0
dtype: int64

### Imputar valores faltantes

In [5]:
categorical = ["Adherencia", "DBT", "Sexo", "Fuma"]
numerical = ["tpo_programa", "TAS", "Peso", "Altura", "IMC", "Edad", "ICC", "tas_basal", "Adherencia_Acumulada", "Adherencia_Total"]

In [6]:
from statistics import mean, median, mode

for cat in categorical:
    tesis[cat].fillna(mode(tesis[tesis["tpo_programa"] == 0][cat]), inplace=True)

for num in numerical:
    tesis[num].fillna(np.mean(tesis[tesis["tpo_programa"] == 0][num]), inplace=True)

In [7]:
tesis["Adherencia_lag1"] = tesis.groupby("idPaciente")["Adherencia"].shift(1).fillna(0)
tesis["Adherencia_lag2"] = tesis.groupby("idPaciente")["Adherencia"].shift(2).fillna(0)
tesis["Adherencia_Acumulada_lag1"] = tesis.groupby("idPaciente")["Adherencia_Acumulada"].shift(1).fillna(0)
tesis["TAS_lag1"] = tesis.groupby("idPaciente")["TAS"].shift(1).fillna(tesis["tas_basal"])
tesis["TAS_lag2"] = tesis.groupby("idPaciente")["TAS"].shift(2).fillna(tesis["tas_basal"])

In [8]:
from sklearn.preprocessing import PolynomialFeatures

columns = tesis.columns

poly = PolynomialFeatures(interaction_only=True, degree=3)
tesis = poly.fit_transform(tesis)
new_columns = poly.get_feature_names(columns)
new_columns = [column.replace(' ', '*') for column in new_columns]

tesis = pd.DataFrame(tesis, columns=new_columns)

tesis['Intercept'] = 1

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
def model(data, effects):
    mixed = sm.MixedLM(endog=data['TAS'], exog=data[effects], exog_re=data[['Intercept', 'tpo_programa']], groups=data['idPaciente']).fit(reml=False)
    return mixed

def model_gee(effects):
    mixed = sm.GEE(endog=tesis['TAS'], exog=tesis[effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente'])
    print(mixed.summary())
    return mixed

### Modelo base

In [10]:
fixed_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    # "DBT",
    # "IMC",
    # "Sexo*Edad",
    # "DBT*Sexo",
    # "IMC*Sexo",
    # "DBT*Edad",
    # "IMC*Edad",
    # "IMC*DBT",
    "tpo_programa",
    # "tpo_programa*Sexo",
    # "tpo_programa*Edad",
    # "tpo_programa*DBT",
    # "tpo_programa*IMC",
    # "tpo_programa*Sexo*Edad",
    # "tpo_programa*DBT*Sexo",
    # "tpo_programa*IMC*Sexo",
    # "tpo_programa*DBT*Edad",
    # "tpo_programa*IMC*Edad",
    # "tpo_programa*IMC*DBT",
]

In [11]:
mod = model(tesis, fixed_effects)
print(mod.summary())
print("AIC:", mod.aic)
print("BIC:", mod.bic)

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  ML         
No. Groups:              560          Scale:                   115.2028   
Min. group size:         7            Log-Likelihood:          -15427.7627
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    121.311    2.355 51.520 0.000 116.696 125.925
Sexo                           3.911    0.756  5.174 0.000   2.429   5.392
Edad                           0.158    0.039  4.074 0.000   0.082   0.235
tpo_programa                  -0.332    0.10

### Probar efectos aleatorios

In [22]:
print(mixed_time.summary())

            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  TAS        
No. Observations:    3920     Method:              REML       
No. Groups:          560      Scale:               148.7790   
Min. group size:     7        Log-Likelihood:      -15661.8792
Max. group size:     7        Converged:           Yes        
Mean group size:     7.0                                      
--------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------
Intercept        121.941    1.636 74.516 0.000 118.733 125.148
Sexo               3.899    0.526  7.414 0.000   2.868   4.929
Edad               0.148    0.027  5.466 0.000   0.095   0.201
tpo_programa      -0.332    0.122 -2.719 0.007  -0.571  -0.093
tpo_programa Var   3.017    0.025                             



In [14]:
mixed_both = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)
mixed_intercept = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept']], groups=tesis['idPaciente']).fit(reml=True)
mixed_time = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)

print(mixed_both.summary())
print((-2*mixed_intercept.llf)-(-2*mixed_both.llf))
print((-2*mixed_time.llf)-(-2*mixed_both.llf))
# Ambos efectos aleatorios son significativos

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  REML       
No. Groups:              560          Scale:                   115.2019   
Min. group size:         7            Log-Likelihood:          -15430.8279
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    121.310    2.361 51.384 0.000 116.683 125.938
Sexo                           3.911    0.758  5.160 0.000   2.425   5.396
Edad                           0.158    0.039  4.063 0.000   0.082   0.235
tpo_programa                  -0.332    0.10

### Selección de modelos

In [13]:
def contrast(fixed_effects, contrast, inplace=True):
    from scipy.stats.distributions import chi2

    # Agregar variables si no están en el modelo
    for x in contrast:
        if x not in fixed_effects:
            fixed_effects.append(x)

    fixed_effects_aux = fixed_effects
    fixed_effects_reduced = [x for x in fixed_effects if x not in contrast]
    mixed_complete = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    mixed_reduced = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects_reduced], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    print(mixed_complete.summary())
    chi_value = (-2*mixed_reduced.llf)-(-2*mixed_complete.llf)
    p_value = chi2.sf(chi_value, len(contrast))
    print('\n', contrast, 'p-value =', p_value)
    if inplace == False:
        return fixed_effects_aux
    elif p_value > 0.05 and inplace == True:
        print('\nSe remueven los efectos de', contrast)
        return fixed_effects_reduced
    else:
        print('\nEl modelo continua igual')
        return fixed_effects

In [14]:
def auto_select_covs(
        data: pd.DataFrame,
        effects: List[str],
        ignore_covs: Optional[List[str]],
        threshold: float = 0.05
    ) -> sm.MixedLM:
    if not ignore_covs:
        ignore_covs = []
    optimized = False
    removed_covs = []
    while not optimized:
        mixed = model(data, effects)
        p_values = mixed.pvalues.to_dict()
        for cov in ignore_covs:
            p_values.pop(cov)
        max_p_value_cov = max(p_values, key=p_values.get)
        max_p_value = p_values[max_p_value_cov]
        if max_p_value >= threshold and max_p_value_cov:
            print(f"Removing {max_p_value_cov} with p-value {max_p_value}")
            effects.remove(max_p_value_cov)
            removed_covs.append(max_p_value_cov)
            continue
        for cov in removed_covs:
            mixed = model(data, effects + [cov])
            p_values = mixed.pvalues.to_dict()
            p = p_values[cov]
            if p < threshold:
                print(f"Adding {cov} with p-value {p}")
                effects.append(cov)
                removed_covs.remove(cov)
                continue
        optimized = True
    mixed = model(data, effects)
    print(mixed.summary())
    return mixed

In [15]:
mod = model(tesis, fixed_effects)

In [16]:
mixed = auto_select_covs(tesis, fixed_effects, ignore_covs=["Intercept", "tpo_programa"])
mixed.aic, mixed.bic

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  ML         
No. Groups:              560          Scale:                   115.2028   
Min. group size:         7            Log-Likelihood:          -15427.7627
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    121.311    2.355 51.520 0.000 116.696 125.925
Sexo                           3.911    0.756  5.174 0.000   2.429   5.392
Edad                           0.158    0.039  4.074 0.000   0.082   0.235
tpo_programa                  -0.332    0.10

(30871.525376116784, 30921.71615157906)

In [17]:
best_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    "tpo_programa",
]

In [22]:
mixed = model(tesis, best_effects + ["tpo_programa*Adherencia", "tpo_programa*Adherencia_Total"])
print(mixed.summary())
round(mixed.aic, 3), round(mixed.bic, 3)

                   Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       TAS        
No. Observations:       3920          Method:                   ML         
No. Groups:             560           Scale:                    113.5247   
Min. group size:        7             Log-Likelihood:           -15393.6515
Max. group size:        7             Converged:                Yes        
Mean group size:        7.0                                                
---------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------
Intercept                     120.832    2.330 51.855 0.000 116.265 125.400
Sexo                            3.754    0.747  5.023 0.000   2.289   5.219
Edad                            0.167    0.039  4.333 0.000   0.091   0.242
tpo_programa                   

(30807.303, 30870.042)

In [19]:
# mixed = sm.MixedLM(endog=tesis['TAS'], exog=tesis[["Intercept", "tpo_programa", "Adherencia", "tpo_programa*Adherencia"]], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
# print(mixed.summary())

In [20]:
# mixed = sm.MixedLM.from_formula(data=tesis, formula="TAS ~ tpo_programa + tpo_programa*Adherencia", groups="idPaciente", re_formula="~tpo_programa").fit(reml=False)
# print(mixed.summary())